In [1]:
!which python  # MUST RUN ON NLP env (for pytorch)


/Users/mcirtain/miniconda3/envs/nlp/bin/python


In [4]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
import json
%matplotlib inline
from urllib.request import urlopen
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si
import requests

In [5]:
url_chris = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/%23chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.json'
#url= 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/project_data.2023-03-11_13.11.55.783862.json'
df = pd.read_json(url_chris)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5070 entries, 0 to 5069
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   conversation_id                  5070 non-null   int64              
 1   lang                             5070 non-null   object             
 2   reply_settings                   5070 non-null   object             
 3   created_at                       5070 non-null   datetime64[ns, UTC]
 4   text                             5070 non-null   object             
 5   author_id                        5070 non-null   int64              
 6   referenced_tweets                4708 non-null   object             
 7   id                               5070 non-null   int64              
 8   edit_history_tweet_ids           5070 non-null   object             
 9   public_metrics.retweet_count     5070 non-null   int64              
 10  publi

In [6]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mcirtain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
df.head()

,conversation_id,lang,reply_settings,created_at,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,[1633954063934009354],8,0,0,0,0,NaN,None,NaN,None
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,[1633954058212876289],616,0,0,0,0,NaN,None,NaN,None
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,[1633951267423768578],616,0,0,0,0,NaN,None,NaN,None
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,[1633950853626318850],14,0,0,0,0,NaN,None,NaN,None
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,[1633950824664645634],8,0,0,0,0,NaN,None,NaN,None


In [8]:
sia = si()
df['polarity_score'] = df.text.apply(lambda x: sia.polarity_scores(x))

In [9]:
df_polar = df[['text','polarity_score']].copy()
df_polar.head()

,text,polarity_score
0,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,"{'neg': 0.292, 'neu': 0.613, 'pos': 0.095, 'co..."
1,RT @1_ofakindnocap: Chris Rock: “we all been c...,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp..."
2,RT @1_ofakindnocap: Chris Rock: “we all been c...,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp..."
3,RT @rolandsmartin: Working out and watching th...,"{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'comp..."
4,RT @rolandsmartin: Workout done. I’ll have a f...,"{'neg': 0.0, 'neu': 0.863, 'pos': 0.137, 'comp..."


In [10]:
print(df_polar.values[:1])

[['RT @Infantry_bucky: He’s lucky a #ChrisRockLive was the worse thing that happened to him .. and he’s still crying !! What a strange charact…'
  {'neg': 0.292, 'neu': 0.613, 'pos': 0.095, 'compound': -0.6988}]]


In [11]:
df1 = pd.DataFrame(df, columns = ["text", "public_metrics.retweet_count", "public_metrics.reply_count",	"public_metrics.like_count", "public_metrics.quote_count", "public_metrics.impression_count", 'polarity_score'])

df1 = df1.rename(columns= {"public_metrics.retweet_count":'retweet', "public_metrics.reply_count":"reply",	"public_metrics.like_count":"like", "public_metrics.quote_count":'quote', "public_metrics.impression_count":"impression"})
df1


,text,retweet,reply,like,quote,impression,polarity_score
0,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,8,0,0,0,0,"{'neg': 0.292, 'neu': 0.613, 'pos': 0.095, 'co..."
1,RT @1_ofakindnocap: Chris Rock: “we all been c...,616,0,0,0,0,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp..."
2,RT @1_ofakindnocap: Chris Rock: “we all been c...,616,0,0,0,0,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp..."
3,RT @rolandsmartin: Working out and watching th...,14,0,0,0,0,"{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'comp..."
4,RT @rolandsmartin: Workout done. I’ll have a f...,8,0,0,0,0,"{'neg': 0.0, 'neu': 0.863, 'pos': 0.137, 'comp..."
...,...,...,...,...,...,...,...
5065,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
5066,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
5067,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
5068,RT @Callme_Tylor: Now this is funny!!!!!! @Mar...,90,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [12]:
df_like = df1.sort_values("like", ascending = False)
df_like = df_like.head(300)
df_like

,text,retweet,reply,like,quote,impression,polarity_score
1153,#MeghanMarkle must be really worried about wha...,50,13,534,1,10103,"{'neg': 0.168, 'neu': 0.832, 'pos': 0.0, 'comp..."
3918,If I slapped someone at work because I don’t l...,7,13,127,1,10629,"{'neg': 0.102, 'neu': 0.768, 'pos': 0.131, 'co..."
390,Shout out to @ohsnapjbsmoove for flying out to...,9,3,66,1,26315,"{'neg': 0.0, 'neu': 0.824, 'pos': 0.176, 'comp..."
2261,SHOCK! Chris Rock HELPED Samantha Markle's Law...,8,3,53,1,5751,"{'neg': 0.295, 'neu': 0.705, 'pos': 0.0, 'comp..."
25,He’s lucky a #ChrisRockLive was the worse thin...,8,7,32,1,832,"{'neg': 0.302, 'neu': 0.599, 'pos': 0.099, 'co..."
...,...,...,...,...,...,...,...
3239,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
3238,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
3237,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
3236,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."


In [13]:
df_retweet = df1.sort_values("retweet", ascending = False)
df_retweet = df_retweet.head(300)
df_retweet

,text,retweet,reply,like,quote,impression,polarity_score
5069,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2496,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2484,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
4238,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2487,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
...,...,...,...,...,...,...,...
2831,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2833,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2834,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."
2835,RT @sagesurge: Marlon Wayans roasting Chris Ro...,9130,0,0,0,0,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp..."


In [14]:
# verification
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9737, 0.0740, 0.7321],
        [0.5260, 0.4024, 0.4277],
        [0.3952, 0.7998, 0.4132],
        [0.1966, 0.7806, 0.4129],
        [0.2772, 0.9962, 0.5291]])


In [19]:
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
nltk.download('stopwords')
import gzip
import gensim 
import logging
logging.basicConfig(format= '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import multiprocessing #import process
from gensim.models.word2vec import Word2Vec
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mcirtain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# Try to fix the following block...
from bs4 import BeautifulSoup

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['text'] = df['text'].apply(clean_text)

In [23]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['text'] = df['text'].apply(clean_text)


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

### Need Help I guess

In [32]:
# Try to fix next cell:
from transformers import AutoConfig, AutoModel, AutoTokenizer

"""
Repo id must use alphanumeric chars 
or '-', '_', '.', 
'--' and '..' are forbidden, 
'-' and '.' cannot start or end the name, 
max length is 96: '/twitter-roberta-base-sentiment'.
"""
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)



In [33]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
#MODEL = f"/twitter-roberta-base-{task}"  # mc change
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"


tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [68]:
text = "Good night 😊"
tweet = preprocess(text)
print(f'preprocessed: {tweet}')
encoded_input = tokenizer(text, return_tensors='pt')
print(f'Encoded input: {encoded_input}')
output = model(**encoded_input)
print(f'output: {output}')
scores = output[0][0].detach().numpy()
print(f'scores: {scores}')
scores = softmax(scores)
print(f'softmax scores: {scores}')
rankiing = np.argsort(scores)
print(f'ranking: {ranking}')
ranking = ranking[:: -1]
print(f'ranking: {ranking}')
for i in range(scores.shape[0]):
    print(f'i: {i}')
    l = labels[ranking[i]]
    print(f'label l: {l}')
    s = scores[ranking[i]]
    print(f'score s: {s}')
    
print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")   ##### WARNING - Not consistent

row = {'text' : tweet, l: [np.round(float(s),4)]}  ##### WARNING - Not consistent
print(f'row: {row}')


preprocessed: Good night 😊
Encoded input: {'input_ids': tensor([[    0, 12350,   363, 17841, 27969,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
output: SequenceClassifierOutput(loss=None, logits=tensor([[-2.4362,  0.5167,  2.2755]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
scores: [-2.4362073   0.51666594  2.2755494 ]
softmax scores: [0.00760987 0.14581212 0.84657806]
ranking: [0 1 2]
ranking: [2 1 0]
i: 0
label l: positive
score s: 0.846578061580658
i: 1
label l: neutral
score s: 0.14581212401390076
i: 2
label l: negative
score s: 0.00760986702516675
tweet : Good night 😊 3) negative 0.0076
row: {'text': 'Good night 😊', 'negative': [0.0076]}


In [109]:
text = "Good night 😊"
tweet = preprocess(text)
print(f'preprocessed: {tweet}')
encoded_input = tokenizer(text, return_tensors='pt')
print(f'Encoded input: {encoded_input}')
output = model(**encoded_input)
print(f'output: {output}')
scores = output[0][0].detach().numpy()
print(f'scores: {scores}')
scores = softmax(scores)
print(f'softmax scores: {scores}')
print(f'Max score and location:', scores.max(), scores.argmax())
sentiments = ['negative', 'neutral', 'positive']
print(f'\nRoBERTa Prediction: {sentiments[scores.argmax()]}, Probability: {scores.max()}')
rankiing = np.argsort(scores)
print(f'ranking: {ranking}')
ranking = ranking[:: -1]
print(f'ranking: {ranking}')
for i in range(scores.shape[0]):
    print(f'i: {i}')
    l = labels[ranking[i]]
    print(f'label l: {l}')
    s = scores[ranking[i]]
    print(f'score s: {s}')
    
# print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")   ##### WARNING - Not consistent

# row = {'text' : tweet, l: [np.round(float(s),4)]}  ##### WARNING - Not consistent
# print(f'row: {row}')
row = {'text': tweet, sentiments[scores.argmax()]: [np.round(scores.max(),4)]}
print(f'row: {row}')


preprocessed: Good night 😊
Encoded input: {'input_ids': tensor([[    0, 12350,   363, 17841, 27969,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
output: SequenceClassifierOutput(loss=None, logits=tensor([[-2.4362,  0.5167,  2.2755]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
scores: [-2.4362073   0.51666594  2.2755494 ]
softmax scores: [0.00760987 0.14581212 0.84657806]
Max score and location: 0.84657806 2

RoBERTa Prediction: positive, Probability: 0.846578061580658
ranking: [2 1 0]
ranking: [0 1 2]
i: 0
label l: negative
score s: 0.00760986702516675
i: 1
label l: neutral
score s: 0.14581212401390076
i: 2
label l: positive
score s: 0.846578061580658
tweet : Good night 😊 3) positive 0.8466
row: {'text': 'Good night 😊', 'positive': [0.8466]}


In [131]:
%%time

# reimplementation...
text = "Good night 😊"
#sentiment_df = pd.DataFrame(columns = ['text'])
sentiments = ['negative', 'neutral', 'positive']

df['roberta_negative'] = None
df['roberta_neutral'] = None
df['roberta_positive'] = None
df['roberta_label'] = None
df['roberta_score'] = None
#for tweet in df['text']:
for idx, row in df.iterrows():
    # classify each tweet, row by row
    encoded_input = tokenizer(row['text'], return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # update our data
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_label'] = sentiments[scores.argmax()]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_argmax'] = scores.max()
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_negative'] = scores[0]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_neutral'] = scores[1]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_positive'] = scores[2]
    
    if idx % 500 == 0:
        print(f'Through {idx} rows')
    
df.head()


Through 0 rows
Through 500 rows
Through 1000 rows
Through 1500 rows
Through 2000 rows
Through 2500 rows
Through 3000 rows
Through 3500 rows
Through 4000 rows
Through 4500 rows
Through 5000 rows
CPU times: user 4min 29s, sys: 1min 25s, total: 5min 54s
Wall time: 2min 55s


,conversation_id,lang,reply_settings,created_at,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,...,geo.place_id,withheld.copyright,withheld.country_codes,polarity_score,roberta_label,roberta_score,roberta_negative,roberta_neutral,roberta_positive,roberta_argmax
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry_bucky hes lucky a #chrisrocklive ...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,[1633954063934009354],8,...,None,NaN,None,"{'neg': 0.292, 'neu': 0.613, 'pos': 0.095, 'co...",negative,None,0.730327,0.228264,0.041409,0.730327
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt 1_ofakindnocap chris rock we all been chea...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,[1633954058212876289],616,...,None,NaN,None,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp...",negative,None,0.875976,0.117147,0.006877,0.875976
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt 1_ofakindnocap chris rock we all been chea...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,[1633951267423768578],616,...,None,NaN,None,"{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'comp...",negative,None,0.875976,0.117147,0.006877,0.875976
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working out and watching the...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,[1633950853626318850],14,...,None,NaN,None,"{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'comp...",neutral,None,0.007859,0.686881,0.30526,0.686881
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done ill have a few ...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,[1633950824664645634],8,...,None,NaN,None,"{'neg': 0.0, 'neu': 0.863, 'pos': 0.137, 'comp...",neutral,None,0.010401,0.872722,0.116877,0.872722


In [38]:
text = "Good night 😊"
sentiment_df = pd.DataFrame(columns = ['text'])

df['roberta_label'] = None
df['roberta_score'] = None
for tweet in df['text']:
  tweet = preprocess(tweet)

  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)



  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")
    row = {'text' : tweet, l: [np.round(float(s),4)]}
    #Append row to the sentiment data frame
    sentiment_df = sentiment_df.append(row, ignore_index = True)  



KeyboardInterrupt: 

In [ ]:
# Can we vectorize this?


In [130]:
df.tail(3)

,conversation_id,lang,reply_settings,created_at,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,...,geo.place_id,withheld.copyright,withheld.country_codes,polarity_score,roberta_label,roberta_score,roberta_negative,roberta_neutral,roberta_positive,roberta_argmax
5067,1632968499302899712,en,everyone,2023-03-07 04:56:43+00:00,rt sagesurge marlon wayans roasting chris roc...,71163570,"[{'type': 'retweeted', 'id': '1632525319973289...",1632968499302899712,[1632968499302899713],9130,...,None,NaN,None,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp...",None,None,None,None,None,NaN
5068,1632968487302750208,en,everyone,2023-03-07 04:56:41+00:00,rt callme_tylor now this is funny marlonwaya...,1340866792789581824,"[{'type': 'retweeted', 'id': '1632371347941740...",1632968487302750208,[1632968487302750210],90,...,None,NaN,None,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",None,None,None,None,None,NaN
5069,1632968431317254144,en,everyone,2023-03-07 04:56:27+00:00,rt sagesurge marlon wayans roasting chris roc...,3244783356,"[{'type': 'retweeted', 'id': '1632525319973289...",1632968431317254144,[1632968431317254144],9130,...,None,NaN,None,"{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp...",None,None,None,None,None,NaN


In [134]:
df.to_csv('../../data/roberta_on_chris.tsv', sep='\t')

In [135]:
# Try to run on all of our data:
url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/normalized_annotated_all_2023-04-07_12.43.49.255973.tsv'

df = pd.read_csv(url, sep='\t')
df.shape

(30611, 37)

In [136]:
%%time

# reimplementation...
#text = "Good night 😊"
#sentiment_df = pd.DataFrame(columns = ['text'])
sentiments = ['negative', 'neutral', 'positive']

df['clean_text'] = df['text'].apply(clean_text)

df['roberta_negative'] = None
df['roberta_neutral'] = None
df['roberta_positive'] = None
df['roberta_label'] = None
df['roberta_score'] = None

for idx, row in df.iterrows():
    # classify each tweet, row by row
    encoded_input = tokenizer(row['clean_text'], return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # update our data
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_label'] = sentiments[scores.argmax()]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_argmax'] = scores.max()
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_negative'] = scores[0]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_neutral'] = scores[1]
    df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_positive'] = scores[2]
    
    if idx % 500 == 0:
        print(f'Through {idx} rows')
    
df.head()

Through 0 rows
Through 500 rows
Through 1000 rows
Through 1500 rows
Through 2000 rows
Through 2500 rows
Through 3000 rows
Through 3500 rows
Through 4000 rows
Through 4500 rows
Through 5000 rows
Through 5500 rows
Through 6000 rows
Through 6500 rows
Through 7000 rows
Through 7500 rows
Through 8000 rows
Through 8500 rows
Through 9000 rows
Through 9500 rows
Through 10000 rows
Through 10500 rows
Through 11000 rows
Through 11500 rows
Through 12000 rows
Through 12500 rows
Through 13000 rows
Through 13500 rows
Through 14000 rows
Through 14500 rows
Through 15000 rows
Through 15500 rows
Through 16000 rows
Through 16500 rows
Through 17000 rows
Through 17500 rows
Through 18000 rows
Through 18500 rows
Through 19000 rows
Through 19500 rows
Through 20000 rows
Through 20500 rows
Through 21000 rows
Through 21500 rows
Through 22000 rows
Through 22500 rows
Through 23000 rows
Through 23500 rows
Through 24000 rows
Through 24500 rows
Through 25000 rows
Through 25500 rows
Through 26000 rows
Through 26500 row

,Unnamed: 0.1,Unnamed: 0,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,normalized_AFINN_scores,normalized_SentiWordNet_scores,our_label,annotator,roberta_negative,roberta_neutral,roberta_positive,roberta_label,roberta_score,roberta_argmax
0,0,0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry_bucky hes lucky a #chrisrocklive ...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",...,0.448980,0.444444,NaN,NaN,0.730327,0.228264,0.041409,negative,None,0.730327
1,1,1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt 1_ofakindnocap chris rock we all been chea...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.367347,0.333333,NaN,NaN,0.875976,0.117147,0.006877,negative,None,0.875976
2,2,2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt 1_ofakindnocap chris rock we all been chea...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.367347,0.333333,NaN,NaN,0.875976,0.117147,0.006877,negative,None,0.875976
3,3,3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working out and watching the...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",...,0.489796,0.412698,NaN,NaN,0.007859,0.686881,0.30526,neutral,None,0.686881
4,4,4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done ill have a few ...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",...,0.489796,0.412698,NaN,NaN,0.010401,0.872722,0.116877,neutral,None,0.872722


In [ ]:
df.to_csv('roberta_all.tsv', sep='\t')

In [ ]:
df.to_json('../../data/roberta_all.json')